In [ ]:
--Austin's Shit

In [1]:
require('hdf5')
require 'nn'
require 'optim'

In [2]:
f = hdf5.open("SST1.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
--X_valid = f:read("valid_input"):all()
--Y_valid = f:read("valid_output"):all()
--X_test = f:read("test_input"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

# Doc by word

In [3]:
--DOC BY WORD (vincent's stuff)
--feature weight: counts
function createDocWordMatrix(vocab, max_sent_len, sparseMatrix)
    docword = torch.zeros(sparseMatrix:size(1), vocab)
    for i=1,sparseMatrix:size(1) do
        for j=1, max_sent_len do
            local idx = (sparseMatrix[i][j])
            if idx ~= 0 then
                docword[i][idx] = 1 + docword[i][idx]
            end
        end        
    end
    return docword
end

function onehotencode(classes, target)
    onehot = torch.zeros(target:size(1), classes)
    for i=1,target:size(1) do
        onehot[i][target[i]] = 1
    end
    return onehot
end

In [4]:
X_train=createDocWordMatrix(nfeatures, 53, X_train)

In [129]:
function linear(b, W, x)
    --weight matrix W (dout x din), bias b (dout x 1), input matrix x (din x n)
    local B = b:clone()
    local B = B:expand(B,B:size(1),x:size(1))
    --print("WX^T+B")
    --print(W)
    --print(x:t())
    --print(B)
    return ((W*x:t()):add(B))
end
    
function second_max(Y_pred, Y_true)
    --Y_pred is a nx5 tensor
    local second_maxes = torch.zeros(Y_pred:size(1),1)
    local indices = second_maxes:clone()
    local Y_prime = Y_pred:clone()
    for i = 1, Y_pred:size(1) do
        Y_prime[i][Y_true[i]] = -100
        second_maxes[i], indices[i] = torch.max(Y_prime[i],1)
        if indices[i] == 1 then
            print('hi')
        end
    end
    return second_maxes, indices
end

function correct_val(Y_pred, Y_true)
    return_val = torch.zeros(Y_true:size(1),1)
    for i = 1, Y_true:size(1) do
        return_val[i] = Y_pred[i][Y_true[i]]
    end
    return return_val
end


function hinge_grad(W,x,b,Y_true,lambda)
    local Y_pred = linear(b,W,x):t()
    local condition = torch.Tensor(Y_true:size(1),1)
    local correct = correct_val(Y_pred,Y_true) 
    local second, s_indices = second_max(Y_pred,Y_true)
    local condition = correct - second
    local W_grad = torch.zeros(W:size())
    local W_temp = torch.zeros(W:size())
    local b_grad = torch.zeros(b:size())
    local b_temp = torch.zeros(b:size())
    for i = 1, condition:size(1) do
        if condition[i][1] < 1 then
            for j = 1, W:size(1) do
                if j == Y_true[i] then
                    b_temp[j] = -1
                    W_temp[j] = x[i]*(-1)
                elseif j == s_indices[i] then
            
                    b_temp[j] = 1
                    W_temp[j] = x[i]
                else
                    b_temp[j] = 0
                    W_temp[j] = W_temp[j]*0
                end
            end
            b_grad:add(b_temp)
            W_grad:add(W_temp)
        end
    end
    return W_grad+2*lambda*torch.sum(W),b_grad+2*lambda*torch.sum(b_grad)
end

function hinge_loss(Y_pred, Y_true, W,lambda)
    --Y_pred is a nx5 tensor, Y_true is an nx1 tensor
    --print(W)
    local ones_vector = torch.ones(Y_true:size(1),1)
    --print("ones vector")
    --print(ones_vector)
    local loss_vector = nn.ReLU():forward(ones_vector+second_max(Y_pred, Y_true)-correct_val(Y_pred,Y_true))
    --print("loss vector")
    --print(loss_vector)
    return loss_vector:sum(1) + lambda*torch.norm(W)
    
end

function obj(params,x,y,lambda)
    --bsize = 1000
    --local idx = torch.randperm(X:size(1)):sub(1,bsize)
    --local x = torch.Tensor(bsize, X:size(2))
    --local y = torch.Tensor(bsize, 1)
    --for i=1,bsize do
    --   x[i] = X[idx[i]]
    --    y[i] = Y[idx[i]]
    --end
    
    --y = y:squeeze()

    local w = params:sub(1,params:size(1),1,params:size(2)-1)
   
    local b = params:sub(1,params:size(1),params:size(2),params:size(2))
    local ypred = linear(b,w,x)
    --print("ypred")
    --print(ypred)
    local sloss = hinge_loss(ypred:t(),y,w,lambda)
    --print("loss")
    ----print(sloss)
    local w1,b1 = hinge_grad(w,x,b,y,lambda)
    --print("weight after hinge grad")
    --print(w1)
    local grads = torch.zeros(params:size())
    grads[{{},{1,params:size(2)-1}}] = w1
    grads[{{},{params:size(2),params:size(2)}}] = b
    print(sloss)
    return sloss, grads
end


function fit(X, Y, iter)
    local W0 = torch.randn(3,X:size(2))
    local b0 = torch.randn(3,1)
    local W = torch.cat(W0, b0, 2)
    
    local func = function(W)
        loss, grads = obj(W, X, Y, 1)
        return loss[1][1], grads
    end
    local state = {learningRate=0.1, maxIter=iter}
    W, f_hist, currentFuncEval = optim.lbfgs(func, W, state)
    
    b = W:sub(1, W:size(1), W:size(2), W:size(2))
    
    --coefficients
    W = W:sub(1, W:size(1), 1, W:size(2)-1)
    
    return W, b
end
    
function predict(X, W, b)
    local b = b:t()
    return (X*W:t()):add(b:expand(b, X:size(1), b:size(2)))
end

function predict_score(ypred, ytrue)
    local c = 0
    for i=1,ypred:size(1) do
        if ypred[i][1] == ytrue[i][1] then
            c = c + 1       
        end
    end
    return c/ypred:size(1)
end

In [130]:
X_train = torch.Tensor{{1,0,0,1},{0,1,0,1},{0,0,1,1}}
Y_train = torch.Tensor{1,2,3}
W, b = fit(X_train, Y_train, 1000)



 11.5496
[torch.DoubleTensor of size 1x1]



 11.5288
[torch.DoubleTensor of size 1x1]

 11.3629
[torch.DoubleTensor of size 1x1]

 11.1877
[torch.DoubleTensor of size 1x1]

 11.0030
[torch.DoubleTensor of size 1x1]

 10.8087
[torch.DoubleTensor of size 1x1]



 10.6046
[torch.DoubleTensor of size 1x1]

 10.3906
[torch.DoubleTensor of size 1x1]

 10.1661
[torch.DoubleTensor of size 1x1]

 9.9303
[torch.DoubleTensor of size 1x1]



 9.6811
[torch.DoubleTensor of size 1x1]

 9.4157
[torch.DoubleTensor of size 1x1]

 9.1288
[torch.DoubleTensor of size 1x1]

 8.8118
[torch.DoubleTensor of size 1x1]



 8.4488
[torch.DoubleTensor of size 1x1]

 8.0085
[torch.DoubleTensor of size 1x1]

 7.4186
[torch.DoubleTensor of size 1x1]



 6.4678
[torch.DoubleTensor of size 1x1]

 6.0899
[torch.DoubleTensor of size 1x1]

 6.1039
[torch.DoubleTensor of size 1x1]

 5.9628
[torch.DoubleTensor of size 1x1]



 7.8382
[torch.DoubleTensor of size 1x1]

 7.4228
[torch.DoubleTensor of size 1x1]

 6.3461
[torch.DoubleTensor of size 1x1]

 125.1056
[torch.DoubleTensor of size 1x1]



 113.7609
[torch.DoubleTensor of size 1x1]



In [123]:
Y_pred = predict(X_train, W, b)
_, Y_pred = torch.max(Y_pred, 2)
print(Y_pred)

 1
 1
 1
[torch.LongTensor of size 3x1]



In [27]:

acc_score = predict_score(Y_pred, Y_train)
print(acc_score)

[string "function linear(b, W, x)..."]:128: attempt to index a number value
stack traceback:
	[string "function linear(b, W, x)..."]:128: in function 'predict_score'
	[string "Y_pred = predict(X_train, W, b)..."]:3: in main chunk
	[C]: in function 'xpcall'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/Vincent/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0106babbd0: 

In [31]:
Y_train = Y_train:reshape(Y_train:size(1), 1)

In [32]:
Y_train:size()

 156817
      1
[torch.LongStorage of size 2]



In [24]:
Y_train = Y_train:reshape(Y_train:size,)

[string "Y_train = Y_train:reshape(Y_train:size,)..."]:1: function arguments expected near ',': 

In [28]:
Y_train = Y_train:squeeze()

In [29]:
Y_train:size()

 156817
[torch.LongStorage of size 1]

